# check drift example

In [51]:
%matplotlib widget

In [62]:
import numpy as np
import matplotlib.pyplot as plt

import spikeinterface.full as si

from pathlib import Path

import probeinterface as pi

## path

In [53]:

base_folder = Path('/home/arthur/Documents/SpikeSorting/Test_20210518/') 

data_folder = base_folder / 'raw_awake'




In [54]:
recording = si.read_spikeglx(data_folder, stream_id='imec0.ap')

probe = pi.read_spikeglx(data_folder / 'raw_awake_01_g0_t0.imec0.ap.meta')

#recording = recording.frame_slice(0, 3000000)
#print(recording)


recording = recording.set_probe(probe)
print(recording)





384
ChannelSliceRecording: 384 channels - 1 segments - 30.0kHz - 3881.358s


In [55]:
recording_f = si.bandpass_filter(recording, freq_min=300, freq_max=6000)
print(recording_f)


BandpassFilterRecording: 384 channels - 1 segments - 30.0kHz - 3881.358s


In [56]:
si.plot_timeseries(recording_f, time_range=(50, 55), order_channel_by_depth=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## detect peaks

In [57]:
noise_levels = si.get_noise_levels(recording_f)
#print(noise_levels)
fig, ax = plt.subplots()
ax.hist(noise_levels)





Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([  1.,   0.,   1.,  44., 180.,  87.,  29.,  18.,  17.,   7.]),
 array([ 4.2650918 ,  5.09582783,  5.92656387,  6.75729991,  7.58803595,
         8.41877198,  9.24950802, 10.08024406, 10.91098009, 11.74171613,
        12.57245217]),
 <BarContainer object of 10 artists>)

In [58]:
from probeinterface.plotting import plot_probe
fig, ax = plt.subplots()
poly_contact, poly_probe = plot_probe(probe, contacts_values=noise_levels, 
    probe_shape_kwargs={'color':'w'},
    show_channel_on_click=True,
    ax=ax,
    )
poly_contact.set_clim(0, 10)
fig.colorbar(poly_contact)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
#from spikeinterface.sortingcomponents import detect_peaks

#peaks = detect_peaks(recording_f, method='locally_exclusive', 
#        peak_sign='neg', detect_threshold=5, n_shifts=5, 
#        local_radius_um=100,
#        noise_levels=noise_levels,
#        random_chunk_kwargs={},
#        outputs='numpy_compact',
#        n_jobs=1, progress_bar=True, chunk_size=30000, )

peaks = np.load('peaks.npy')



In [64]:
peaks

array([(        5,  77, -41.53484646, 0),
       (      124,  89, -76.14118224, 0),
       (      335, 266, -70.90998763, 0), ...,
       (116442587, 302, -70.87296255, 0),
       (116442588, 117, -46.27979285, 0),
       (116442591,  83, -58.17060557, 0)],
      dtype=[('sample_ind', '<i8'), ('channel_ind', '<i8'), ('amplitude', '<f8'), ('segment_ind', '<i8')])

In [65]:
si.plot_peak_activity_map(recording_f, peaks=peaks)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [69]:
si.plot_drift_over_time(recording_f, peaks=peaks, bin_duration_s=30,
    mode='heatmap', weight_with_amplitudes=False)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [68]:
si.plot_drift_over_time(recording_f, peaks=peaks, bin_duration_s=30,
    mode='scatter', weight_with_amplitudes=False)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …